Transfer learning adalah suatu teknik atau metode yang memanfaatkan model yang sudah dilatih terhadap suatu dataset untuk menyelesaikan permasalahan lain yang serupa dengan cara menggunakannya sebagai starting point, memodifikasi dan mengupdate parameternya sehingga sesuai dengan dataset yang baru.

# ImageNet

<img src="img/imageNet.jpg">

Sebelum melangkah lebih lanjut saya akan bahas sedikit tentang ImageNet. ImageNet adalah sebuah dataset yang terdiri dari 1.200.000 gambar untuk training dan 100.000 untuk testing. Dataset ini terdiri dari 1000 classes jadi untuk setiap class ada 1.200 gambar.

Tiap tahun ada semacam challenge untuk mencari model buatan siapa yang mempunyai tingkat akurasi tertinggi.

<img src="img/imageNet_result.png">

Sebenarnya ImageNet challenge ini sudah dimulai sejak 2010, saya kurang paham algoritma dan model apa yang digunakan pada rentang 2010–2011. Namun hasil yang paling standout adalah AlexNet pada 2012. AlexNet adalah model pertama yang menggunakan Convolutional Neural Network (CNN). Untuk detail lebih lanjut mungkin bisa dibaca sendiri papernya di [sini](https://www.cs.toronto.edu/~kriz/imagenet_classification_with_deep_convolutional.pdf).

<img src="img/more_deeper_better.png">

Seiring dengan perkembangan teknologi tiap tahun, jumlah layer yang digunakan juga mengalami kenaikan yang hasilnya bisa dibilang setara dengan tingkat akurasi yang dihasilkan.

<img src="img/alexNet_architecture.png">

Gambar diatas adalah ilustrasi arsitektur AlexNet. Sama seperti arsitektur model yang sudah kita coba pada Part-7 lalu. AlexNet terbagi menjadi 2 bagian yaitu Feature Extraction layer dan Fully-Connected Layer.

# Gender Classification

Kali ini kita akan melakukan percobaan untuk melakukan klasifikasi jenis kelamin seseorang berdasarkan foto mereka.

Dataset yang digunakan saya kumpulkan dari [UI Faces API](https://uifaces.co/api-docs) dan [Random User Generator](https://randomuser.me/photos). Ada 800 foto untuk training data dan 240 foto untuk testing yang dibagi menjadi 2 classes yaitu male dan female. Gambar dari Random User Generator mempunyai ukuran yang sama yaitu 128x128 pixels namun foto dari UI Faces mempunyai ukuran yang berbeda-beda. Sehingga kita harus melakukan pre-processing terlebih dahulu untuk mendapatkan ukuran foto yang seragam sehingga dapat lebih mudah saat melakukan training.

[Download Dataset](https://www.dropbox.com/s/9e1ix1xunqlq6vw/dataset.zip?dl=0)

<img src="img/sample_dataset.png">

Pada percobaan ini kita akan menggunakan 2 buah model. Model pertama kita akan menggunakan arsitektur yang sederhana yang kita kasih nama KorNet dan model kedua adalah VGG-16 Network. Buat yang penasaran tentang VGG secara lengkap, bisa dibaca sendiri papernya di [sini](https://arxiv.org/pdf/1409.1556.pdf).

# KorNet

Arsitektur sederhana kita ini terdiri dari 87.969 buah parameter yang akan diupdate pada saat training. Pada *feature learning layer* terdapat **4 Convolution Layer**, **ZeroPadding Layer** dan **MaxPooling Layer**.

Sedangkan pada *fully-connected layer* terdapat **2 buah layer dengan jumlah neuron masing-masing sebanyak 32 dan 1**. Perlu diingat bahwa layer terakhir adalah output layer. **1 buah layer disini karena kita akan melakukan binary classification, 0 untuk pria dan 1 untuk wanita**. Sehingga **activation function** yang harus kita gunakan pada output layer adalah **sigmoid dengan loss function binary crossentropy**. Kita juga bisa menggunakan 2 neuron pada output layer, menggunakan activation function softmax dan loss function categorical crossentropy seperti pada Part-7.

Boleh dicoba mana yang lebih baik, tapi untuk percobaan kali ini, kita mau coba dengan 1 neuron dan sigmoid function. Berikut adalah contoh arsitektur korNet:

<img src="img/cnn_korNet.jpg">

# Dependencies and Global Variable

Dependencies yang akan kita gunakan hampir sama dengan Part-7. Tapi kali ini kita akan gunakan **ImageDataGenerator**. Dimensi gambar yang digunakan adalah **128 x 128 pixels**. Kita juga gunakan TensorBoard untuk visualisasi pada saat training.

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

# Images Dimensions
img_width, img_height = 128, 128

train_data_dir = 'data/datasets/train'
validation_data_dir = 'data/datasets/validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./graph')

Using TensorFlow backend.


# Data Augmentation

<img src="img/data_augmentation.png">

Seperti yang sudah kita ketahui, untuk mendapatkan performa yang optimal, Deep Learning membutuhkan data yang lebih banyak dibandingkan dengan algoritma ML yang lain.

Dari dataset yang telah kita kumpulkan hanya terdapat 400 foto pria dan 400 foto wanita. Jumlah data tersebut masih kurang mencukupi untuk mendapatkan performa yang optimal.

Untuk itu kita perlu meng-augmentasi data tersebut. Data Augmentation adalah sebuah teknik memanipulasi sebuah data tanpa kehilangan inti atau esensi dari data tersebut. Untuk data berupa Image, hal yang biasanya dilakukan adalah **rotate**, **flip**, **crop**, dll.

In [2]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Train Data Generator
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

Found 800 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


Pada percobaan kita kali ini, kita akan melakukan **shear**, **zoom** dan **flip** sedangkan parameter rescale yang kita gunakan adalah membagi nilai RGB dari 0-255 dengan 255, sehingga kita mendapatkan nilai RGB pada rentang 0-1. Untuk **data testing kita hanya melakukan rescale saja**.

Method `flow_from_directory` dari `ImageDataGenerator` kita gunakan untuk mengubah data yang berupa "raw image" menjadi sebuah dataset yang akan kita gunakan untuk training dan testing, tentu saja dataset yang telah kita augmentasi tadi.

# KorNet Model

Kali ini kita akan melakukan klasifikasi terhadap 2 class atau biasa disebut binary classification. 0 untuk class pertama dan 1 untuk class kedua. Jika dilihat dari karakteristik outputnya, kita bisa gunakan "Sigmoid" sebagai activation function pada output layer dan untuk semua hidden layer kita gunakan ReLU.

Karena ini adalah **binary classification**, **loss function** yang kita gunakan adalah **binary_crossentropy** dan **Adam** sebagai *optimizer*-nya.

In [3]:
# Feature Extraction Layer
inputs = Input(shape=(img_width, img_height, 3))
conv_layer = Conv2D(16, (5, 5), strides=(3,3), activation='relu')(inputs) 
conv_layer = ZeroPadding2D(padding=(1,1))(conv_layer) 
conv_layer = Conv2D(32, (5, 5), strides=(3,3), activation='relu')(conv_layer) 
conv_layer = MaxPooling2D((2, 2))(conv_layer) 
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer) 
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer)

# Flatten Layer
flatten = Flatten()(conv_layer) 

# Fully Connected Layer
fc_layer = Dense(32, activation='relu')(flatten)
outputs = Dense(1, activation='sigmoid')(fc_layer)

model = Model(inputs=inputs, outputs=outputs)

# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

# Print Model Summary
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 16)        1216      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 44, 44, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2

In [4]:
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size, 
                    callbacks=[callbacks])

model.save_weights('kornet.h5')




Epoch 1/50
50/50 [==============================] - 5s 104ms/step - loss: 0.6919 - accuracy: 0.5200 - val_loss: 0.6878 - val_accuracy: 0.5958

Epoch 2/50
50/50 [==============================] - 6s 127ms/step - loss: 0.6871 - accuracy: 0.5387 - val_loss: 0.6895 - val_accuracy: 0.5833
Epoch 3/50
50/50 [==============================] - 6s 111ms/step - loss: 0.6686 - accuracy: 0.6325 - val_loss: 0.6303 - val_accuracy: 0.6417
Epoch 4/50
50/50 [==============================] - 5s 94ms/step - loss: 0.6276 - accuracy: 0.6687 - val_loss: 0.5526 - val_accuracy: 0.6333
Epoch 5/50
50/50 [==============================] - 5s 95ms/step - loss: 0.6086 - accuracy: 0.6737 - val_loss: 0.5375 - val_accuracy: 0.6375
Epoch 6/50
50/50 [==============================] - 5s 94ms/step - loss: 0.5815 - accuracy: 0.7038 - val_loss: 0.6615 - val_accuracy: 0.6750
Epoch 7/50
50/50 [==============================] - 5s 97ms/step - loss: 0.5492 - accuracy: 0.7150 - val_loss: 0.5147 - val_accuracy: 0.6333
Epoch 

Setelah 50 epoch training-testing, kita mendapatkan loss dan accuracy sebesar 0.4847 crossentropy loss dan 76.97% accuracy. Grafik dibawah juga menunjukkan bahwa performa dari model kita ini tidak begitu baik. Terdapat indikasi overfitting, nilai loss dan akurasi yang relatif rendah.

<img src="img/tf_board_kornet.png">